<a href="https://colab.research.google.com/github/sugamkhadka40-rgb/Sugam-Khadka-CPSMA-3933-01/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from math import comb
import random
import numpy as np
from math import comb

1. Random Function Call

In [ ]:
def random_bottle_pull_weight():
    if random.randint(0, 9) == 0:
        return 9
    else:
        return 10

print(f"\nTest Pull Result: {random_bottle_pull_weight()} oz")

2. Theoretical Expected Weight

In [ ]:
E_W_T = (0.1 * 9) + (0.9 * 10)
print(f"\n1. Theoretical Expected Weight: {E_W_T:.2f} oz")
print("   This means the average weight of all 100 bottles is 9.9 oz.")

3. Simulation Expected Weight

In [ ]:
NUM_DRAWS = 1000
weights = [random_bottle_pull_weight() for _ in range(NUM_DRAWS)]
E_W_S = np.mean(weights)
print(f"2. Simulation Average Weight (from {NUM_DRAWS} pulls): {E_W_S:.2f} oz")
print("   The simulation result should be very close to the theoretical 9.9 oz.")

print("\n" + "="*70)
print("### SECTION 2: WATER CASES - EXPECTED PULLS TO FIND THE SHORT BOTTLE")
print("="*70)

4. Theoretical Expected Pulls

In [ ]:
E_P_T = 1 / 0.1
print(f"\n3. Theoretical Pulls to find short bottle: {E_P_T:.0f} pulls")
print("   We expect to make 10 pulls on average before finding the short bottle.")

5. Simulation Pulls

In [ ]:
def simulate_find_short_bottle():
    """Simulates pulling one bottle at a time until the 9 oz bottle is found."""
    pulls = 0
    while random_bottle_pull_weight() == 10:
        pulls += 1
    return pulls + 1

NUM_EXPERIMENTS = 10000
pull_counts = [simulate_find_short_bottle() for _ in range(NUM_EXPERIMENTS)]
E_P_S = np.mean(pull_counts)
print(f"4. Simulation Average Pulls (from {NUM_EXPERIMENTS} runs): {E_P_S:.2f} pulls")
print("   The simulation result agrees well with the theoretical 10 pulls.")

6. Modification Proposal

In [ ]:
print("\n5. Modification Proposal for Real-World Problem:")
print("   The current model is for finding ANY 9 oz bottle (10% chance).")
print("   The REAL problem is finding the ONE case that is short among 10 cases.")
print("   Suggestion: Use a list of 10 items for cases, e.g., `[10, 10, ..., 9]`.")
print("   We would draw cases WITHOUT replacement. The expected draws would be (10+1)/2 = 5.5.")


print("\n" + "="*70)
print("### SECTION 3: PLATES - THEORETICAL PROBABILITIES")
print("="*70)

Plates

In [ ]:
TOTAL, PINK, DRAW = 24, 6, 4
NON_PINK = TOTAL - PINK
COLORS = {'Pink': 6, 'Blue': 6, 'Black': 6, 'Green': 6}
initial_plates = ['Pink'] * 6 + ['Blue'] * 6 + ['Black'] * 6 + ['Green'] * 6

1. theoretical first draw pink

In [ ]:
P_1P = PINK / TOTAL # 6/24
print(f"\n6. Theoretical P(1st draw Pink): {P_1P:.3f}")

2. theoretical one pink in  4 draw

In [ ]:
P_0P = comb(NON_PINK, DRAW) / comb(TOTAL, DRAW)
P_GE1P = 1 - P_0P
print(f"7. Theoretical P(At least 1 Pink in 4 draws): {P_GE1P:.3f}")

3. simulation

In [ ]:
NUM_EXP_PLATES = 100000
GE1P_count = 0
for _ in range(NUM_EXP_PLATES):
    draw_group = random.sample(initial_plates, DRAW)
    if 'Pink' in draw_group:
        GE1P_count += 1
print(f"8. Simulation P(At least 1 Pink in 4 draws - Sim): {GE1P_count / NUM_EXP_PLATES:.3f}")
print("   The simulation closely matches the theoretical probability.")
print("\n" + "="*70)
print("### SECTION 4: PLATES - LIFO STACK AND WASHING SIMULATION")
print("="*70)

4 and 5. LIFO/STACK SETUP and washing/shuffling logic

In [ ]:
def draw_and_wash_sim(stack, num_experiments):
    """Simulates draws with LIFO and shuffling (washing) plates."""
    p_exact_one_pink_count = 0
    p_at_least_one_pink_count = 0

    for _ in range(num_experiments):
        current_stack = stack.copy()
        drawn_plates = [current_stack.pop() for _ in range(DRAW) if current_stack]
        plates_in_total = drawn_plates + current_stack

        if len(current_stack) <= 12:
            random.shuffle(plates_in_total)
            stack[:] = plates_in_total
        else:

            random.shuffle(drawn_plates)
            stack[:] = current_stack + drawn_plates

        pink_count = drawn_plates.count('Pink')
        if pink_count == 1: p_exact_one_pink_count += 1
        if pink_count >= 1: p_at_least_one_pink_count += 1

    return p_exact_one_pink_count / num_experiments, p_at_least_one_pink_count / num_experiments


sim_p_exact_one_wash, sim_p_at_least_one_wash = draw_and_wash_sim(initial_plates.copy(), 10000)
print(f"\n9. P(Exactly 1 Pink, LIFO+Wash - Sim): {sim_p_exact_one_wash:.3f}")
print(f"10. P(At least 1 Pink, LIFO+Wash - Sim): {sim_p_at_least_one_wash:.3f}")
print("\n" + "="*70)
print("### SECTION 5: PLATES - APPEASEMENT (REJECTION) AND LONG-TERM MIGRATION")
print("="*70)

6. rejection function

In [ ]:
def appease_daughter_draw(stack, usage_counts):
    """Draws until at least one pink plate is found. Rejected plates return to the top."""
    rejected = []
    while True:
        draw = [stack.pop() for _ in range(DRAW) if stack]

        if 'Pink' in draw:
            for plate in draw:
                usage_counts[plate] += 1

            random.shuffle(draw)
            stack.extend(rejected)
            stack.extend(draw)
            return
        else:
            rejected.extend(draw)
            if not stack:
                 random.shuffle(rejected)
                 stack.extend(rejected)
                 rejected = []

7. long term simulation

In [ ]:
NUM_DAYS = 1000
unique_plates = [f'{c}_{i+1}' for c, n in COLORS.items() for i in range(n)]
random.shuffle(unique_plates)
daily_stack = unique_plates.copy()
usage_counts = {plate: 0 for plate in unique_plates}
all_pink_stack_count = 0

for day in range(NUM_DAYS):
    appease_daughter_draw(daily_stack, usage_counts)

    if all(p.startswith('Pink') for p in daily_stack[-6:]):
         all_pink_stack_count += 1

8. final result

In [ ]:
usage_analysis = sorted(usage_counts.items(), key=lambda item: item[1], reverse=True)
top_used = usage_analysis[:3]
least_used = usage_analysis[-3:]

print(f"\n11. Days with All Pink at Top (over {NUM_DAYS} days): {all_pink_stack_count}")
print("\n12. Plate Usage Analysis:")
print(f"   Highest Used Plate: {top_used[0][0]} ({top_used[0][1]} uses)")
print(f"   Lowest Used Plate: {least_used[0][0]} ({least_used[0][1]} uses)")
print("\n   Discussion on Migration:")
print("   The process of rejecting non-pink plates and placing them back on top (along with the used pink plates) causes a strong 'migration' of the pink plates to the top of the stack, leading to high usage for those plates and very low usage for the non-pink plates that sink to the bottom.")